<a href="https://colab.research.google.com/github/profteachkids/CHE2064/blob/master/max_entropy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
import numpy as np
import jax 
import jax.numpy as jnp
from scipy.optimize import minimize, Bounds, LinearConstraint, NonlinearConstraint
from jax.config import config
config.update("jax_enable_x64", True)
from plotly.subplots import make_subplots
import plotly.io as pio
pio.templates.default = 'plotly_dark'

In [36]:
def neg_entropy(p):
  return jnp.dot(p,jnp.log(p))

s = jnp.linspace(0,10,11)
p_init = jnp.ones(11)/11

In [37]:
linear = LinearConstraint(jnp.stack([jnp.ones_like(s),s]), [1.,7.], [1.,7.])

In [46]:
def stdev_cons(p):
  avg = jnp.average(s,weights=p)
  var = jnp.average((s-avg)**2, weights=p)
  return jnp.array([jnp.sqrt(var)])

In [47]:
non_linear = NonlinearConstraint(stdev_cons,[2.], [2.], jac=jax.jacobian(stdev_cons),
                                 hess=jax.hessian(lambda p,v: jnp.dot(stdev_cons(p),v)))

In [48]:
res=minimize(neg_entropy, p_init, method='trust-constr', jac=jax.grad(neg_entropy),
         hess=jax.hessian(neg_entropy), tol=1e-6, options=dict(maxiter=100),
         constraints=(linear, non_linear), bounds=Bounds(0.,1.), )

In [50]:
fig = make_subplots()
fig.add_bar(x=s, y=res.x)
fig.update_layout(width=600,height=400)
fig.show()

In [ ]:
stdev(res.x)

DeviceArray(2.76375737, dtype=float64)